In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
group_phase_dataset = pd.read_csv("/home/bawbaw31/Documents/datasets/LOL Worlds 2018 Knockout stage - Player Ratings.csv")

In [4]:
group_phase_dataset[group_phase_dataset["Team"] == "G2"]

,Name,Team,Position,KDA Ratio,Kills(Total),Deaths(Total),Assists(Total),Kill Participation,CS(Per Minute),CS(Total),Minutes Played,Games Played
6,Hjärnan,G2,ADC,4.6,18,11,33,0.61,10.2,2506,246,8
17,Jankos,G2,Jungle,3.0,17,21,45,0.74,4.8,1188,246,8
18,Perkz,G2,Mid,2.8,28,19,26,0.64,9.4,2315,246,8
19,Wadid,G2,Support,2.7,2,20,52,0.64,1.5,359,246,8
28,Wunder,G2,Top,2.2,19,19,22,0.49,9.3,2290,246,8
